In [1]:
#
import numpy as np
import os
import pandas as pd
import pickle
from sklearn import preprocessing
import time

C:\Users\subha\AppData\Local\conda\conda\envs\pytorch\lib\site-packages\sklearn\utils\__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


In [9]:
#
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [4]:
#
data_folder = 'D:\GitHub\IndependentStudy\Data\historical_data1_Q12005'

In [8]:
#
loan_dtypes = {'credit_score': np.int64, 'first_payment_date': np.int64, 'first_time': object, 'mat_date': np.int64, 'msa':np.float64, 'mortgage_insurance':np.float64, 
              'num_units':np.int64, 'occupancy':object, 'CLTV': np.float64, 'DTI': np.float64, 'UPB': np.float64, 'LTV': np.float64, 'original_interest_rate':np.float64,
              'channel': object, 'PPM': object, 'product_type': object, 'state': object, 'property_type': object, 'postal_code': np.float64, 'loan_number': object, 
              'purpose': object, 'original_loan_term': np.float64, 'num_borrowers': np.int64, 'seller': object, 'servicer': object, 'super_conforming_flag': object, 'preharp_seq_num': object}
loan_name = ['credit_score', 'first_payment_date', 'first_time', 'mat_date', 'msa', 'mortgage_insurance', 'num_units', 'occupancy', 'CLTV', 'DTI', 'UPB', 'LTV', 'original_interest_rate', 'channel', 'PPM', 'product_type', 'state', 'property_type', 'postal_code', 'loan_number','purpose', 'original_loan_term', 'num_borrowers', 'seller', 'servicer', 'super_conforming_flag', 'preharp_seq_num']

In [9]:
#
monthly_dtypes = {'loan_number': object, 'monthly_period': np.float64, 'current_UPB': np.float64, 'current_status': object, 'loan_age': np.float64,
                 'remaining_month': np.float64, 'repurchase_flag': object, 'modification_flag': object, 'zero_balance': np.float64, 'zero_balance_date': np.float64,
                 'current_interest_rate': np.float64, 'current_deferred_UPB': np.float64, 'DDLPI': np.float64, 'mi_recovery': np.float64,
                 'net_sales_procedees': object, 'non_mi_recovery': np.float64, 'expenses': np.float64, 'legal_cost': np.float64, 'maintainance_cost': np.float64,
                 'tax': np.float64, 'misc_cost': np.float64, 'actual_loss': np.float64, 'modification_cost': np.float64, 'step_modification_flag': object, 'deferred_payment_mod': object}

monthly_name = ['loan_number', 'monthly_period', 'current_UPB', 'current_status', 'loan_age',
                 'remaining_month', 'repurchase_flag', 'modification_flag', 'zero_balance', 'zero_balance_date',
                 'current_interest_rate', 'current_deferred_UPB', 'DDLPI', 'mi_recovery',
                 'net_sales_procedees', 'non_mi_recovery', 'expenses', 'legal_cost', 'maintainance_cost',
                 'tax', 'misc_cost', 'actual_loss', 'modification_cost', 'step_modification_flag', 'deferred_payment_mod']

In [7]:
%time loan_data = pd.read_pickle(os.path.join(data_folder, 'loan_data1_time_Q12005_np.pickle'))
%time monthly_data = pd.read_pickle(os.path.join(data_folder, 'monthly_data1_time_Q12005_np.pickle'))

Wall time: 1.31 s
Wall time: 1min 12s


In [29]:
#
%time loan_data = pd.read_csv(os.path.join(data_folder, 'historical_data1_Q12005.txt'), delimiter='|', header = None, dtype=loan_dtypes, names=loan_name)
%time monthly_data = pd.read_csv(os.path.join(data_folder, 'historical_data1_time_Q12005.txt'), delimiter='|', header = None, dtype=monthly_dtypes, names=monthly_name)

Wall time: 9.98 s


### One-hot encoding

In [10]:
#
loan_one_hot_columns = ['first_time', 'occupancy', 'channel', 'PPM', 'product_type', 'property_type', 'purpose', 'seller', 'servicer', 'super_conforming_flag', 'preharp_seq_num']

monthly_one_hot_columns = ['repurchase_flag', 'modification_flag', 'step_modification_flag', 'deferred_payment_mod']

In [31]:
#
%time loan_data_temp = pd.get_dummies(loan_data, columns=loan_one_hot_columns)

Wall time: 1.76 s


In [9]:
#
%time monthly_data_temp = pd.get_dummies(monthly_data, columns=monthly_one_hot_columns)

Wall time: 1min 7s


In [38]:
#
print(loan_data.shape)
print(monthly_data.shape)

(351634, 69)
(26391679, 29)


### Creating lookup

In [ ]:
lookup = monthly_data[['loan_number']].groupby(['loan_number']).size().reset_index(name='counts')

In [ ]:
lookup_table = []
curr = 0
for ind, r in lookup.iterrows():
    lookup_table.append((r[0], curr, curr+r[1]))
    curr = curr+r[1]

In [ ]:
lookup_table = np.array(lookup_table)

### Dividing into 100 chunks loans, joining, creating lookup and saving

In [41]:
#
num_chunk = 100
chunk_size = int(loan_data.shape[0] / 100)

In [14]:
rootDir = '../Data'
flag = False
for dirName, subdirList, fileList in os.walk(rootDir):
    print(dirName)
    print(subdirList)
    for f in fileList:
        if 'time' in f:
            monthly_data = pd.read_csv(os.path.join(dirName, f), delimiter='|', header = None, dtype=monthly_dtypes, names=monthly_name)
        else:
            loan_data = pd.read_csv(os.path.join(dirName, f), delimiter='|', header = None, dtype=loan_dtypes, names=loan_name)
            flag = True
    if flag:
        break

../Data
['2014', '2015']
../Data/2014
['historical_data1_Q12014', 'historical_data1_Q22014', 'historical_data1_Q32014', 'historical_data1_Q42014']
../Data/2014/historical_data1_Q12014
[]


In [16]:
loan_data.head()

,credit_score,first_payment_date,first_time,mat_date,msa,mortgage_insurance,num_units,occupancy,CLTV,DTI,...,property_type,postal_code,loan_number,purpose,original_loan_term,num_borrowers,seller,servicer,super_conforming_flag,preharp_seq_num
0,717,201404,9,204403,39300.0,0.0,1,I,77.0,41.0,...,SF,2800.0,F114Q1000001,P,360.0,2,Other sellers,LAKEVIEWLOANSERVICIN,NaN,NaN
1,813,201405,Y,204404,19780.0,30.0,1,P,95.0,32.0,...,SF,50300.0,F114Q1000002,P,360.0,1,Other sellers,Other servicers,NaN,NaN
2,799,201403,9,204402,NaN,0.0,1,P,79.0,37.0,...,SF,45600.0,F114Q1000003,C,360.0,1,Other sellers,Other servicers,NaN,NaN
3,742,201403,9,204402,NaN,0.0,1,P,78.0,45.0,...,SF,42100.0,F114Q1000004,C,360.0,1,Other sellers,Other servicers,NaN,NaN
4,725,201403,9,204402,17860.0,0.0,1,P,80.0,20.0,...,SF,65200.0,F114Q1000005,N,360.0,2,Other sellers,"PNCBANK,NATL",NaN,NaN


In [17]:
l = [['we', 4, 6], ['tr', 5, 1]]
l

[['we', 4, 6], ['tr', 5, 1]]

In [18]:
pd.DataFrame(l)

,0,1,2
0,we,4,6
1,tr,5,1


In [43]:
#
group_name = 'Q1/'
for i in range(0, num_chunk):
    curr_loans = loan_data.iloc[i*chunk_size:min((i+1)*chunk_size, loan_data.shape[0]), :]
    df = pd.merge(curr_loans, monthly_data, on='loan_number')
    
    # Create Lookup
    lookup = df[['loan_number']].groupby(['loan_number']).size().reset_index(name='counts')
    lookup_table = []
    curr = 0
    for ind, r in lookup.iterrows():
        lookup_table.append((r[0], curr, curr+r[1]))
        curr = curr+r[1]
    lookup_table = pd.DataFrame(np.array(lookup_table))
    
    df.to_hdf(os.path.join(data_folder, 'Loan_Data.h5'), key=group_name+'X_'+str(i), format='t', complevel=9)
    lookup_table.to_hdf(os.path.join(data_folder, 'Loan_data.h5'), key=group_name+'lookup_'+str(i), format='t', complevel=9)
    

1.139484167098999
0.08643150329589844
1.1762776374816895
0.05512237548828125
0.8045287132263184
0.032903432846069336
0.8777880668640137
0.05741119384765625
1.1112720966339111
0.04189300537109375
0.7760803699493408
0.051168203353881836
1.1109113693237305
0.0359036922454834
0.5811624526977539
0.02692580223083496
1.0741286277770996
0.05584287643432617
0.7370274066925049
0.07081246376037598
1.068134069442749
0.0458829402923584
1.0093011856079102
0.06781625747680664
1.1080400943756104
0.04787302017211914
1.1120247840881348
0.024933338165283203
0.9813776016235352
0.031917572021484375
1.6785097122192383
0.03590512275695801
0.8028531074523926
0.03390812873840332
1.0761139392852783
0.08277153968811035
0.8477420806884766
0.0648653507232666
0.9155495166778564
0.016959667205810547
1.2317054271697998
0.03989410400390625
1.2785792350769043
0.04089188575744629
0.8675594329833984
0.03986525535583496
0.6690754890441895
0.06087851524353027
0.7320425510406494
0.06682372093200684
1.1047418117523193
0.0392

In [44]:
df.shape

(260654, 97)

## Write Functions

In [16]:
start_t = time.time()
with open(os.path.join(data_folder, 'monthly_data1_time_Q12005_np.pickle'), 'wb') as handle:
    pickle.dump(monthly_data_temp, handle)
end_t = time.time()
print(end_t - start_t)

63.54519581794739


In [15]:
start_t = time.time()
with open(os.path.join(data_folder, 'loan_data1_time_Q12005_np.pickle'), 'wb') as handle:
    pickle.dump(loan_data, handle)
end_t = time.time()
print(end_t - start_t)

1.0314583778381348


In [ ]:
start_t = time.time()
with open(os.path.join(data_folder, 'lookup1_time_Q12005.pickle'), 'wb') as handle:
    pickle.dump(lookup_table, handle)
end_t = time.time()
print(end_t - start_t)

In [ ]:
# %time monthly_data.to_hdf(os.path.join(data_folder, 'monthly_data1_time_Q12005.h5'), key='monthly_data')

In [ ]:
# import h5py

In [ ]:
# h5f = h5py.File(os.path.join(data_folder, 'monthly_data1_time_Q12005.h5'), 'w')
# h5f.create_dataset('dataset_1', data=)

In [2]:
a= [[3,4,3], [1,2,4], [0,9,5], [5,8,7], [6,1,6], [0,1,2]]
a = np.array(a)
a

array([[3, 4, 3],
       [1, 2, 4],
       [0, 9, 5],
       [5, 8, 7],
       [6, 1, 6],
       [0, 1, 2]])

In [4]:
np.delete(a, [1,3,4], 0)

array([[3, 4, 3],
       [0, 9, 5],
       [0, 1, 2]])

In [8]:
list(a[:, 1] - 1)

[3, 1, 8, 7, 0, 0]